<a href="https://colab.research.google.com/github/monya-9/deep-learning-practice/blob/main/14_gpt2_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-2로 문장 생성 실험, Sampling 방식 변경

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [2]:
# 1. 모델 & 토크나이저 불러오기
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
# 2. 입력 문장
prompt = "Artificial intelligence will change the future of"
inputs = tokenizer(prompt, return_tensors="pt").input_ids

In [4]:
# 3. 다양한 Sampling 방식 실험
# (1) Greedy Search
greedy_output = model.generate(
    inputs,
    max_length=50
)

# (2) Sampling (temperature 낮음 → 덜 창의적)
sample_output_temp_low = model.generate(
    inputs,
    max_length=50,
    do_sample=True,
    temperature=0.7,
    num_return_sequences=2
)

# (3) Sampling (temperature 높음 → 더 창의적)
sample_output_temp_high = model.generate(
    inputs,
    max_length=50,
    do_sample=True,
    temperature=1.5,
    num_return_sequences=2
)

# (4) Top-k Sampling (상위 k개 단어에서만 선택)
sample_output_topk = model.generate(
    inputs,
    max_length=50,
    do_sample=True,
    top_k=50,
    num_return_sequences=2
)

# (5) Nucleus Sampling (top-p, 확률 누적 0.9까지 고려)
sample_output_topp = model.generate(
    inputs,
    max_length=50,
    do_sample=True,
    top_p=0.9,
    num_return_sequences=2
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask

- Greedy Search: 항상 가장 확률 높은 단어만 선택 → 무난하지만 반복/단조로움.

- Temperature:

  - 0.7 → 보수적, 안정적인 결과

  - 1.5 → 창의적, 예측 불가한 결과

- Top-k Sampling: 확률 상위 k개만 후보로 → 불필요한 단어 제거, 다양성 확보.

- Nucleus Sampling (Top-p): 확률 누적이 p(예: 0.9) 미만인 단어만 후보 → 상황에 맞는 균형 잡힌 결과.

In [5]:
# 4. 결과 출력
print("=== Greedy ===")
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

print("\n=== Sampling (temperature=0.7) ===")
for out in sample_output_temp_low:
    print(tokenizer.decode(out, skip_special_tokens=True))

print("\n=== Sampling (temperature=1.5) ===")
for out in sample_output_temp_high:
    print(tokenizer.decode(out, skip_special_tokens=True))

print("\n=== Top-k Sampling (k=50) ===")
for out in sample_output_topk:
    print(tokenizer.decode(out, skip_special_tokens=True))

print("\n=== Nucleus Sampling (top-p=0.9) ===")
for out in sample_output_topp:
    print(tokenizer.decode(out, skip_special_tokens=True))

=== Greedy ===
Artificial intelligence will change the future of the world.

The future of the world is not yet clear. But it is clear that AI will change the world.

The future of the world is not yet clear. But it is clear

=== Sampling (temperature=0.7) ===
Artificial intelligence will change the future of the world, and we can't wait to see where it goes next.

The article was originally published on The Conversation. Read the original article.
Artificial intelligence will change the future of life, not just the present.

=== Sampling (temperature=1.5) ===
Artificial intelligence will change the future of healthcare in ways that many experts view as devastating by comparison."


Read or Share this story: http://usat.ly/1K1WcPV
Artificial intelligence will change the future of computing, just long enough to fill all computing jobs (most in computers). It is not just computer programming and algorithms we need to pay attention to—it makes the game that much much better."

At

=== Top